In [35]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn import preprocessing
### IMPORT DATA, PERFORM PRE-PROCESSING
rawdata = pd.read_csv("finalProjectSample.csv")

data=rawdata.drop(['Unnamed: 0'],axis=1)
# convert termination_process_ts to datetime, then extract only the date (ignore the time)
data['FiredDate'] = pd.to_datetime(data.TERMINATION_PROCESS_TS, infer_datetime_format=True).dt.date
# do the same for report_dt and hire_process_ts
data['ReportDate'] = pd.to_datetime(data.REPORT_DT, infer_datetime_format=True).dt.date
data['hiredDate'] = pd.to_datetime(data.HIRE_PROCESS_TS, infer_datetime_format=True).dt.date
# generate a dummy determining if individual was fired today
# changed to inequality from equality, since reporting only occurs every 2 days
#     and some individuals quit on the day between reports
data['FiredToday'] = [int(data.FiredDate[x]<=data.ReportDate[x]) for x in range(len(data))]

# process veteran_flg
data['Vet'] = [int(data.VETERAN_FLG[x]=='Y') for x in range(len(data))]

# process tenure variable as timedelta type
data['daysWorked'] = (data.ReportDate - data.hiredDate).astype('timedelta64[D]')

# process report date as datetime in order to extract day, month, and year info
data['ReportDate'] = data['ReportDate'] .astype('datetime64[ns]')
data['rDay'] = data.ReportDate.dt.dayofweek
data['rMonth'] = data.ReportDate.dt.month
data['rYear'] = data.ReportDate.dt.year

# generate dummy variables based on available categorical variables
data = pd.concat([data, pd.get_dummies(data.Max_Students, prefix='hireType', drop_first=True)], axis=1)
data = pd.concat([data, pd.get_dummies(data.STATE_CD, prefix='state', drop_first=True)], axis=1)
# Changed to drop_first=False to get LongHaul dummy
data = pd.concat([data, pd.get_dummies(data.Driver_type, prefix='driver')], axis=1)
data = pd.concat([data, pd.get_dummies(data.rDay, prefix='day', drop_first=True)], axis=1)
data = pd.concat([data, pd.get_dummies(data.rMonth, prefix='month', drop_first=True)], axis=1)
data = pd.concat([data, pd.get_dummies(data.rYear, prefix='year', drop_first=True)], axis=1)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
#Create descriptive statistics summary
summ = data.describe().transpose()
print(summ)
summ.to_csv('DescriptiveStatistics.csv')
descriptiveStatistics = pd.read_csv('DescriptiveStatistics.csv').round(0)

                               count           mean            std       min  \
Unique_ID                   126312.0  149729.280963   22438.711960  100277.0   
Age                         126312.0      39.227856      11.078927      19.0   
Max_Students                126312.0     559.012604     465.757234       0.0   
Retention (Days)            126312.0    1261.527321    1725.607792       0.0   
SEGMENT_LINEHAUL_MILES_QTY  126312.0  217930.612618  236426.355871       0.0   
...                              ...            ...            ...       ...   
month_11                    126312.0       0.068006       0.251758       0.0   
month_12                    126312.0       0.070460       0.255922       0.0   
year_2015                   126312.0       0.282040       0.449994       0.0   
year_2016                   126312.0       0.271194       0.444578       0.0   
year_2017                   126312.0       0.167854       0.373738       0.0   

                                 25%   

In [37]:
data['TERMINATION_TYPE_CD'].value_counts()

# Filtering VQUIT, ACTIVE
data=data[(data['TERMINATION_TYPE_CD']=='VQUIT') | (data['TERMINATION_TYPE_CD']=='ACTIVE')]

In [38]:
# Dependent Variable creation
#===============================
data['DaysuntilQuit'] = (data.FiredDate - pd.to_datetime(data.REPORT_DT, infer_datetime_format=True).dt.date).astype('timedelta64[D]')
data=data[data['DaysuntilQuit']>0]
data=data.reset_index()

data['Quitwithin30days']=np.where(data['DaysuntilQuit']<=30,1,0) #<- dependent variable

In [39]:
data['percent_miles_pay']=data['percent_miles_pay'].fillna(0)
data['PayDay']=np.where(data['percent_miles_pay']==0,0,1)

In [40]:
# Variables from last n days - change TIMEWINDOW variable to change days

TIMEWINDOW=30
data.loc[:,'Milesinlast30days']=data.groupby(['Unique_ID'])['Miles_Quantity_Total'].rolling(TIMEWINDOW,min_periods=2).sum().reset_index(0,drop=True)
data.loc[:,'Milesinlast30days']=data.loc[:,'Milesinlast30days'].fillna(0)
data.loc[:,'PayDaysinlast30days']=data.groupby(['Unique_ID'])['PayDay'].rolling(TIMEWINDOW,min_periods=2).sum().reset_index(0,drop=True)
data.loc[:,'PayDaysinlast30days']=data.loc[:,'PayDaysinlast30days'].fillna(0)
data.loc[:,'RegionalDrivinglast30days']=data.groupby(['Unique_ID'])['driver_Regional'].rolling(TIMEWINDOW,min_periods=2).sum().reset_index(0,drop=True)
data.loc[:,'RegionalDrivinglast30days']=data.loc[:,'RegionalDrivinglast30days'].fillna(0)
data.loc[:,'SpecRegionalDrivinglast30days']=data.groupby(['Unique_ID'])['driver_Specialized Regional'].rolling(TIMEWINDOW,min_periods=2).sum().reset_index(0,drop=True)
data.loc[:,'SpecRegionalDrivinglast30days']=data.loc[:,'SpecRegionalDrivinglast30days'].fillna(0)
data.loc[:,'TeamDrivinglast30days']=data.groupby(['Unique_ID'])['driver_Team'].rolling(TIMEWINDOW,min_periods=2).sum().reset_index(0,drop=True)
data.loc[:,'TeamDrivinglast30days']=data.loc[:,'TeamDrivinglast30days'].fillna(0)
data.loc[:,'TraineeDrivinglast30days']=data.groupby(['Unique_ID'])['driver_Trainee'].rolling(TIMEWINDOW,min_periods=2).sum().reset_index(0,drop=True)
data.loc[:,'TraineeDrivinglast30days']=data.loc[:,'TraineeDrivinglast30days'].fillna(0)
data.loc[:,'LongHaulDrivinglast30days']=data.groupby(['Unique_ID'])['driver_Long Haul'].rolling(TIMEWINDOW,min_periods=2).sum().reset_index(0,drop=True)
data.loc[:,'LongHaulDrivinglast30days']=data.loc[:,'LongHaulDrivinglast30days'].fillna(0)
data.loc[:,'ActiveProdLast30days']=data.groupby(['Unique_ID'])['ACTIVE_PRODUCTION_STATE_CNT'].rolling(TIMEWINDOW,min_periods=2).sum().reset_index(0,drop=True)
data.loc[:,'ActiveProdLast30days']=data.loc[:,'ActiveProdLast30days'].fillna(0)
data.loc[:,'AvailProdLast30days']=data.groupby(['Unique_ID'])['AVAILABLE_PRODUCTION_STATE_CNT'].rolling(TIMEWINDOW,min_periods=2).sum().reset_index(0,drop=True)
data.loc[:,'AvailProdLast30days']=data.loc[:,'AvailProdLast30days'].fillna(0)
data.loc[:,'InvProdLast30days']=data.groupby(['Unique_ID'])['INVALID_PRODUCTION_STATE_CNT'].rolling(TIMEWINDOW,min_periods=2).sum().reset_index(0,drop=True)
data.loc[:,'InvProdLast30days']=data.loc[:,'InvProdLast30days'].fillna(0)
#data.loc[:,'DaysWithCompanyLast30days']=data.groupby(['Unique_ID'])['daysWorked'].rolling(TIMEWINDOW,min_periods=1).max().reset_index(0,drop=True)
#data.loc[:,'DaysWithCompanyLast30days']=data.loc[:,'DaysWithCompanyLast30days'].fillna(0)


In [41]:
data[data['Unique_ID']==100277][['Unique_ID','ReportDate','Milesinlast30days','PayDaysinlast30days','RegionalDrivinglast30days','SpecRegionalDrivinglast30days','TeamDrivinglast30days','TraineeDrivinglast30days','LongHaulDrivinglast30days','ActiveProdLast30days','InvProdLast30days']].head(60)


,Unique_ID,ReportDate,Milesinlast30days,PayDaysinlast30days,RegionalDrivinglast30days,SpecRegionalDrivinglast30days,TeamDrivinglast30days,TraineeDrivinglast30days,LongHaulDrivinglast30days,ActiveProdLast30days,InvProdLast30days
0,100277,2014-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100277,2014-01-03,505.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0
2,100277,2014-01-05,505.0,1.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0
3,100277,2014-01-07,505.0,1.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0
4,100277,2014-01-09,714.0,2.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0
5,100277,2014-01-11,714.0,2.0,6.0,0.0,0.0,0.0,0.0,6.0,0.0
6,100277,2014-01-13,923.0,2.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0
7,100277,2014-01-15,923.0,2.0,8.0,0.0,0.0,0.0,0.0,8.0,0.0
8,100277,2014-01-17,1544.0,2.0,9.0,0.0,0.0,0.0,0.0,9.0,0.0
9,100277,2014-01-19,1769.0,2.0,10.0,0.0,0.0,0.0,0.0,10.0,0.0


In [42]:
#Label Encoders
data['GENDER_CD'] = [int(data.GENDER_CD[x]=='M ') for x in range(len(data))]
data['GENDER_CD']

0         1
1         1
2         1
3         1
4         1
         ..
115157    1
115158    1
115159    1
115160    1
115161    1
Name: GENDER_CD, Length: 115162, dtype: int64

In [43]:
le1 = preprocessing.LabelEncoder() 
le1=le1.fit(data['REASON_CD'])
list(le1.classes_) 
data['REASON_CD']=le1.transform(data['REASON_CD'])

In [44]:
le2 = preprocessing.LabelEncoder() 
le2=le2.fit(data['REHIRED'])
list(le2.classes_) 
data['REHIRED']=le2.transform(data['REHIRED'])

In [45]:
le3 = preprocessing.LabelEncoder() 
le4=le3.fit(data['STATE_CD'])
list(le3.classes_) 
data['STATE_CD']=le3.transform(data['STATE_CD'])

In [46]:
data['CSA Count']=data['CSA Count'].fillna(0)

In [47]:
vardata=data[['Quitwithin30days','Age','GENDER_CD','Vet','hireType_500','hireType_1000','TERMINATION_TYPE_CD','REASON_CD','CSA Count','REHIRED','STATE_CD','Milesinlast30days','PayDaysinlast30days','RegionalDrivinglast30days','SpecRegionalDrivinglast30days','TeamDrivinglast30days','TraineeDrivinglast30days','LongHaulDrivinglast30days','ActiveProdLast30days','InvProdLast30days']]
vardata

,Quitwithin30days,Age,GENDER_CD,Vet,hireType_500,hireType_1000,TERMINATION_TYPE_CD,REASON_CD,CSA Count,REHIRED,STATE_CD,Milesinlast30days,PayDaysinlast30days,RegionalDrivinglast30days,SpecRegionalDrivinglast30days,TeamDrivinglast30days,TraineeDrivinglast30days,LongHaulDrivinglast30days,ActiveProdLast30days,InvProdLast30days
0,0,53,1,0,0,1,VQUIT,17,1.0,0,41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,53,1,0,0,1,VQUIT,17,1.0,0,41,505.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0
2,0,53,1,0,0,1,VQUIT,17,1.0,0,41,505.0,1.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0
3,0,53,1,0,0,1,VQUIT,17,1.0,0,41,505.0,1.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0
4,0,53,1,0,0,1,VQUIT,17,1.0,0,41,714.0,2.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115157,1,45,1,1,0,0,VQUIT,16,0.0,0,38,3441.0,3.0,30.0,0.0,0.0,0.0,0.0,19.0,0.0
115158,1,45,1,1,0,0,VQUIT,16,0.0,0,38,3441.0,3.0,30.0,0.0,0.0,0.0,0.0,18.0,0.0
115159,1,45,1,1,0,0,VQUIT,16,0.0,0,38,3302.0,3.0,30.0,0.0,0.0,0.0,0.0,17.0,0.0
115160,1,45,1,1,0,0,VQUIT,16,0.0,0,38,3302.0,3.0,30.0,0.0,0.0,0.0,0.0,16.0,0.0


In [48]:
#Create descriptive statistics summary
summ = vardata.describe().transpose()
print(summ)
summ.to_csv('DescriptiveStatistics.csv')
descriptiveStatistics = pd.read_csv('DescriptiveStatistics.csv').round(0)


                                  count         mean          std  \
Quitwithin30days               115162.0     0.110045     0.312947   
Age                            115162.0    39.062408    10.828179   
GENDER_CD                      115162.0     0.906436     0.291222   
Vet                            115162.0     0.182048     0.385886   
hireType_500                   115162.0     0.115403     0.319509   
hireType_1000                  115162.0     0.496605     0.499991   
REASON_CD                      115162.0     8.778773     8.865800   
CSA Count                      115162.0     0.484769     1.058233   
REHIRED                        115162.0     0.163335     0.369673   
STATE_CD                       115162.0    18.277140    14.120609   
Milesinlast30days              115162.0  6253.391693  3515.917441   
PayDaysinlast30days            115162.0     3.005827     1.800008   
RegionalDrivinglast30days      115162.0    10.331568    13.409429   
SpecRegionalDrivinglast30days  115

In [49]:
traindata=vardata[vardata['TERMINATION_TYPE_CD']=='VQUIT'].drop('TERMINATION_TYPE_CD',axis=1)

In [50]:
testdata=vardata=vardata[vardata['TERMINATION_TYPE_CD']=='ACTIVE'].drop('TERMINATION_TYPE_CD',axis=1)

In [51]:
traindata

,Quitwithin30days,Age,GENDER_CD,Vet,hireType_500,hireType_1000,REASON_CD,CSA Count,REHIRED,STATE_CD,Milesinlast30days,PayDaysinlast30days,RegionalDrivinglast30days,SpecRegionalDrivinglast30days,TeamDrivinglast30days,TraineeDrivinglast30days,LongHaulDrivinglast30days,ActiveProdLast30days,InvProdLast30days
0,0,53,1,0,0,1,17,1.0,0,41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,53,1,0,0,1,17,1.0,0,41,505.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0
2,0,53,1,0,0,1,17,1.0,0,41,505.0,1.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0
3,0,53,1,0,0,1,17,1.0,0,41,505.0,1.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0
4,0,53,1,0,0,1,17,1.0,0,41,714.0,2.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115157,1,45,1,1,0,0,16,0.0,0,38,3441.0,3.0,30.0,0.0,0.0,0.0,0.0,19.0,0.0
115158,1,45,1,1,0,0,16,0.0,0,38,3441.0,3.0,30.0,0.0,0.0,0.0,0.0,18.0,0.0
115159,1,45,1,1,0,0,16,0.0,0,38,3302.0,3.0,30.0,0.0,0.0,0.0,0.0,17.0,0.0
115160,1,45,1,1,0,0,16,0.0,0,38,3302.0,3.0,30.0,0.0,0.0,0.0,0.0,16.0,0.0


In [52]:
traindata.columns

Index(['Quitwithin30days', 'Age', 'GENDER_CD', 'Vet', 'hireType_500',
       'hireType_1000', 'REASON_CD', 'CSA Count', 'REHIRED', 'STATE_CD',
       'Milesinlast30days', 'PayDaysinlast30days', 'RegionalDrivinglast30days',
       'SpecRegionalDrivinglast30days', 'TeamDrivinglast30days',
       'TraineeDrivinglast30days', 'LongHaulDrivinglast30days',
       'ActiveProdLast30days', 'InvProdLast30days'],
      dtype='object')

In [53]:
testdata

,Quitwithin30days,Age,GENDER_CD,Vet,hireType_500,hireType_1000,REASON_CD,CSA Count,REHIRED,STATE_CD,Milesinlast30days,PayDaysinlast30days,RegionalDrivinglast30days,SpecRegionalDrivinglast30days,TeamDrivinglast30days,TraineeDrivinglast30days,LongHaulDrivinglast30days,ActiveProdLast30days,InvProdLast30days
1240,0,42,0,0,1,0,0,2.0,1,19,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1241,0,42,0,0,1,0,0,2.0,1,19,0.00,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1242,0,42,0,0,1,0,0,2.0,1,19,0.00,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
1243,1,42,0,0,1,0,0,2.0,1,19,0.00,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
1244,1,42,0,0,1,0,0,2.0,1,19,0.00,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114500,1,46,1,0,0,0,0,0.0,0,3,9984.65,6.0,30.0,0.0,0.0,0.0,0.0,30.0,0.0
114501,1,46,1,0,0,0,0,0.0,0,3,10605.40,5.0,30.0,0.0,0.0,0.0,0.0,30.0,0.0
114502,1,46,1,0,0,0,0,0.0,0,3,10627.50,5.0,30.0,0.0,0.0,0.0,0.0,30.0,0.0
114503,1,46,1,0,0,0,0,0.0,0,3,11107.20,5.0,30.0,0.0,0.0,0.0,0.0,30.0,0.0


In [54]:
y_train=traindata['Quitwithin30days']
X_train=traindata.drop('Quitwithin30days',axis=1)
y_test=testdata['Quitwithin30days']
X_test=testdata.drop('Quitwithin30days',axis=1)

In [55]:
from sklearn.linear_model import LogisticRegression
#for Train/Test K-Fold Cross-Validation scoring
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score
#Random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [56]:
rfc= RandomForestClassifier().fit(X_train,y_train)
rfc_pred=rfc.predict(X_test)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [57]:
# Random Forest Classifier
rfc= RandomForestClassifier().fit(X_train,y_train)
rfc_pred=rfc.predict(X_test)
# Checking accuracy
print('Test accuracy score:',accuracy_score(y_test,rfc_pred))

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Test accuracy score: 0.8884607891945713


In [58]:
# Checking f1
print('Test f1 score:',accuracy_score(y_test,rfc_pred))
# Checking recall
print('Test recall score',recall_score(y_test,rfc_pred))

#Checking unique values
predictions=pd.DataFrame(rfc_pred)
predictions[0].value_counts()

Test f1 score: 0.8884607891945713
Test recall score 0.052479338842975204


0    43212
1     2987
Name: 0, dtype: int64

In [59]:
# Logistic Regression
lr = LogisticRegression(solver='liblinear').fit(X_train,y_train)
print('logreg train score:',lr.score(X_train,y_train))
print('logreg test score:',lr.score(X_test,y_test))

logreg train score: 0.8441773124719052
logreg test score: 0.9195653585575445


In [60]:
import numpy as np
import pandas as pd
import patsy as pt
import statsmodels.api as sm

y, x = pt.dmatrices('Quitwithin30days ~ Age + GENDER_CD + Vet + hireType_500 + hireType_1000 + REASON_CD + REHIRED + STATE_CD + Milesinlast30days + PayDaysinlast30days + RegionalDrivinglast30days + SpecRegionalDrivinglast30days + TeamDrivinglast30days + TraineeDrivinglast30days + LongHaulDrivinglast30days + ActiveProdLast30days + InvProdLast30days', data = traindata)

model = sm.Logit(y, x)

reg = model.fit()

print(reg.summary())

Optimization terminated successfully.
         Current function value: 0.382177
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:       Quitwithin30days   No. Observations:                68963
Model:                          Logit   Df Residuals:                    68945
Method:                           MLE   Df Model:                           17
Date:                Mon, 14 Dec 2020   Pseudo R-squ.:                 0.09093
Time:                        05:00:55   Log-Likelihood:                -26356.
converged:                       True   LL-Null:                       -28992.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                        -2.7475      0.083    -33.092      0.

In [61]:
# Marginal Effect
MA = reg.get_margeff(at = 'overall', method = 'dydx')
print(MA.summary())

        Logit Marginal Effects       
Dep. Variable:       Quitwithin30days
Method:                          dydx
At:                           overall
                                   dy/dx    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Age                              -0.0011      0.000     -9.181      0.000      -0.001      -0.001
GENDER_CD                         0.0456      0.005      9.933      0.000       0.037       0.055
Vet                               0.0006      0.004      0.162      0.871      -0.007       0.008
hireType_500                      0.0449      0.004     10.173      0.000       0.036       0.054
hireType_1000                     0.0452      0.004     12.391      0.000       0.038       0.052
REASON_CD                         0.0023      0.000     11.113      0.000       0.002       0.003
REHIRED                           0.0102      0.004      2.653  

In [62]:
pred=pd.DataFrame(rfc_pred,columns=['Quitwithin30days'])
pred

,Quitwithin30days
0,0
1,0
2,0
3,0
4,0
...,...
46194,0
46195,0
46196,0
46197,0


In [65]:
testdatawithID=data[data['TERMINATION_TYPE_CD']=='ACTIVE'][['Unique_ID']]
testdatawithID.reset_index(inplace=True,drop=True)
QuitList=pd.concat([pred,testdatawithID],axis=1)
QuitList=QuitList[QuitList['Quitwithin30days']==1][['Unique_ID']].drop_duplicates()

In [66]:
QuitList

,Unique_ID
57,101908
783,102006
933,102984
1108,103270
1785,103999
...,...
40756,178384
41425,179880
43415,181352
43974,182150
